In [1]:
'''
페이지 링크 싹다 가져오는 버전

'''


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

def get_href_links(driver, wait, actions, num_items_to_fetch=100):
    href_links = set()
    while len(href_links) < num_items_to_fetch:
        actions.send_keys(Keys.END)
        time.sleep(2)
        
        for x in range(1, 46):
            try:
                xpath = f'//*[@id="root"]/main/div/section[3]/div[1]/div/div[{x}]/div/div[2]/a[2]'
                element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
                href = element.get_attribute('href')
                href_links.add(href)
            except (NoSuchElementException, TimeoutException):
                continue

        print(f"Current number of unique href links: {len(href_links)}")
        if len(href_links) >= num_items_to_fetch:
            break

    return list(href_links)

def extract_reviews(driver, wait):
    actions = driver.find_element(By.CSS_SELECTOR, 'body')
    time.sleep(1)
    actions.send_keys(Keys.END)
    time.sleep(2)
    actions.send_keys(Keys.END)
    time.sleep(2)
    actions.send_keys(Keys.END)
    time.sleep(1)

    
    reviews = []
    for n in range(3, 10):
        try:
            review_id = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[1]/div/div[1]/p[1]'))).text
            print(f"review_id: {review_id}")
            #//*[@id="style_estimate_list"]/div/div[3]/div[1]/div/div[1]/p[1]
            try:
                weight_height_gender = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[1]/div/div[2]/p[1]'))).text
                print(f"weight_height_gender: {weight_height_gender}")
            except:
                weight_height_gender = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[1]/div/div[2]/p'))).text
                print(f"weight_height_gender (alternative): {weight_height_gender}")
            
            
            top_size = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[1]/span'))).text
            print(f"top_size: {top_size}")
            
            brightness_comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[2]/span'))).text
            print(f"brightness_comment: {brightness_comment}")
            
            color_comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[3]/span'))).text
            print(f"color_comment: {color_comment}")
            
            thickness_comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[4]/span'))).text
            print(f"thickness_comment: {thickness_comment}")
            
            purchased_product_id = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[2]/div[2]/a'))).get_attribute('href')
            print(f"purchased_product_id: {purchased_product_id}")
            
            purchased_size = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[2]/div[2]/p/span'))).text
            print(f"purchased_size: {purchased_size}")
            
            comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[2]'))).text
            print(f"comment: {comment}")

            review = {
                "weight_height_gender": weight_height_gender,
                "review_id": review_id,
                "top_size": top_size,
                "brightness_comment": brightness_comment,
                "color_comment": color_comment,
                "thickness_comment": thickness_comment,
                "purchased_product_id": purchased_product_id,
                "purchased_size": purchased_size,
                "comment": comment
            }
            reviews.append(review)
        except (NoSuchElementException, TimeoutException):
            print(f"Review information not found for element index: {n}")
            continue
    
    return reviews

def get_product_info(driver, wait, href_links):
    products = []
    for index, link in enumerate(href_links):
        driver.get(link)
        time.sleep(2)  # 페이지 로드 대기

        try:
            product_id = f"top{index + 1}"
            
            try:
                product_name = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[3]/h2'))).text
            except (NoSuchElementException, TimeoutException):
                product_name = "N/A"
                print(f"Product name not found for link: {link}")
                
            try:
                category = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[2]/a[1]'))).text
            except (NoSuchElementException, TimeoutException):
                category = "N/A"
                print(f"Category not found for link: {link}")

            try:
                price = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[5]/div/div/span'))).text.strip()
            except (NoSuchElementException, TimeoutException):
                price = "N/A"
                print(f"Price not found for link: {link}")

            try:
                image_element = wait.until(EC.presence_of_element_located((By.XPATH, '//img[@class="sc-1jl6n79-4 AqRjD"]')))
                image_url = image_element.get_attribute('src')
            except (NoSuchElementException, TimeoutException):
                image_url = "N/A"
                print(f"Image URL not found for link: {link}")

            description = link

            # 사이즈 정보 가져오기
            sizes = []
            try:
                ul_element = wait.until(EC.presence_of_element_located((By.XPATH, '//ul[contains(@class, "sc-1sxlp32-1") or contains(@class, "sc-8wsa6t-1 Qtsoe")]')))
                li_elements = ul_element.find_elements(By.TAG_NAME, 'li')
                for li in li_elements:
                    size_text = li.text.strip()
                    if size_text not in ["cm", "MY"]:
                        sizes.append(size_text)
            except (NoSuchElementException, TimeoutException):
                print(f"Size information not found for link: {link}")

            # 리뷰 정보 가져오기
            reviews = extract_reviews(driver, wait)

            product = {
                "product_id": product_id,
                "product_name": product_name,
                "category": category,
                "price": price,
                "image_url": image_url,
                "description": description,
                "size": sizes,
                "reviews": reviews
            }
            products.append(product)
            
            # 데이터 출력
            print(product)

        except Exception as e:
            print(f"Failed to extract data for link: {link} due to {e}")
            continue
    
    return products

def save_to_csv(products, filename="products.csv"):
    df = pd.DataFrame(products)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Data saved to {filename}")

def main():
    URL = "https://www.musinsa.com/categories/item/001?device=mw&sortCode=emt_high"
    option = webdriver.ChromeOptions()
    option.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)
    driver.get(URL)
    time.sleep(3)

    driver.find_element(By.XPATH, '//*[@id="root"]/main/div/div[3]/div/button').click()
    time.sleep(1)

    wait = WebDriverWait(driver, 10)
    actions = driver.find_element(By.CSS_SELECTOR, 'body')

    href_links = get_href_links(driver, wait, actions, num_items_to_fetch=100)
    print(f"Number of unique href links: {len(href_links)}")

    products = get_product_info(driver, wait, href_links)
    print("Products extracted:")
    for product in products:
        print(product)

    save_to_csv(products)

    # driver.quit()

if __name__ == '__main__':
    main()


Current number of unique href links: 45
Current number of unique href links: 90
Current number of unique href links: 135
Number of unique href links: 135
review_id: LV.5 덕배맘
weight_height_gender: 남성 · 172cm · 65kg
top_size: 보통이에요
brightness_comment: 보통이에요
color_comment: 보통이에요
thickness_comment: 보통이에요
purchased_product_id: https://www.musinsa.com/app/goods/2702397/0
purchased_size: 세이지 올리브/M
comment: 아빠 사줬는데 너무 예뻐요ㅠㅠ
재질도 좋고 아빠가 받자마자 비싼거같은데??
라고 했슴다 색감도 핏도 완벽스 ㅎ
review_id: LV.3 정상적인회색코디
weight_height_gender: 여성 · 162cm · 50kg
top_size: 커요
brightness_comment: 보통이에요
color_comment: 보통이에요
thickness_comment: 보통이에요
purchased_product_id: https://www.musinsa.com/app/goods/2702397/0
purchased_size: 페일 스카이/L
comment: 가을이고 쿨톤인데 이런 색감과 두께와 디자인의 니트 하나가 없어서 구매했어용 만족합니다 색감이 예뻐요!
review_id: LV.5 ansinsa
weight_height_gender: 남성 · 176cm · 65kg
top_size: 보통이에요
brightness_comment: 밝아요
color_comment: 선명해요
thickness_comment: 보통이에요
purchased_product_id: https://www.musinsa.com/app/goods/2702397/0
purchased_si

KeyboardInterrupt: 

In [14]:
# 테스트 코드
'''
중간 부분이 안들어와져서 중간 부분부터 시작하는 코드인데 아마 너무 오래 걸려서 페이지 로딩이 느려져 안가져와지는 듯함
중간 부분은 잘가져와진다 (xpath, class name 문제는 아님)

'''
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

def get_href_links(driver, wait, actions, num_items_to_fetch=100):
    href_links = set()
    while len(href_links) < num_items_to_fetch:
        actions.send_keys(Keys.END)
        time.sleep(2)
        
        for x in range(1, 46):
            try:
                xpath = f'//*[@id="root"]/main/div/section[3]/div[1]/div/div[{x}]/div/div[2]/a[2]'
                element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
                href = element.get_attribute('href')
                href_links.add(href)
            except (NoSuchElementException, TimeoutException):
                continue

        print(f"Current number of unique href links: {len(href_links)}")
        if len(href_links) >= num_items_to_fetch:
            break

    return list(href_links)

def extract_reviews(driver, wait):
    actions = driver.find_element(By.CSS_SELECTOR, 'body')
    time.sleep(1)
    actions.send_keys(Keys.END)
    time.sleep(2)
    actions.send_keys(Keys.END)
    time.sleep(2)
    actions.send_keys(Keys.END)
    time.sleep(1)

    
    reviews = []
    for n in range(3, 10):
        try:
            review_id = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[1]/div/div[1]/p[1]'))).text
            print(f"review_id: {review_id}")
            #//*[@id="style_estimate_list"]/div/div[3]/div[1]/div/div[1]/p[1]
            try:
                weight_height_gender = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[1]/div/div[2]/p[1]'))).text
                print(f"weight_height_gender: {weight_height_gender}")
            except:
                weight_height_gender = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[1]/div/div[2]/p'))).text
                print(f"weight_height_gender (alternative): {weight_height_gender}")
            
            
            top_size = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[1]/span'))).text
            print(f"top_size: {top_size}")
            
            brightness_comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[2]/span'))).text
            print(f"brightness_comment: {brightness_comment}")
            
            color_comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[3]/span'))).text
            print(f"color_comment: {color_comment}")
            
            thickness_comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[4]/span'))).text
            print(f"thickness_comment: {thickness_comment}")
            
            purchased_product_id = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[2]/div[2]/a'))).get_attribute('href')
            print(f"purchased_product_id: {purchased_product_id}")
            
            purchased_size = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[2]/div[2]/p/span'))).text
            print(f"purchased_size: {purchased_size}")
            
            comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[2]'))).text
            print(f"comment: {comment}")

            review = {
                "weight_height_gender": weight_height_gender,
                "review_id": review_id,
                "top_size": top_size,
                "brightness_comment": brightness_comment,
                "color_comment": color_comment,
                "thickness_comment": thickness_comment,
                "purchased_product_id": purchased_product_id,
                "purchased_size": purchased_size,
                "comment": comment
            }
            reviews.append(review)
        except (NoSuchElementException, TimeoutException):
            print(f"Review information not found for element index: {n}")
            continue
    
    return reviews

def get_product_info(driver, wait, href_links):
    products = []
    for index, link in enumerate(href_links[63:], start=64):  # 64번째 링크부터 시작
        driver.get(link)
        time.sleep(2)  # 페이지 로드 대기

        try:
            product_id = f"top{index + 1}"
            
            try:
                product_name = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[3]/h2'))).text
            except (NoSuchElementException, TimeoutException):
                product_name = "N/A"
                print(f"Product name not found for link: {link}")
                
            try:
                category = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[2]/a[1]'))).text
            except (NoSuchElementException, TimeoutException):
                category = "N/A"
                print(f"Category not found for link: {link}")

            try:
                price = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[5]/div/div/span'))).text.strip()
            except (NoSuchElementException, TimeoutException):
                price = "N/A"
                print(f"Price not found for link: {link}")

            try:
                image_element = wait.until(EC.presence_of_element_located((By.XPATH, '//img[@class="sc-1jl6n79-4 AqRjD"]')))
                image_url = image_element.get_attribute('src')
            except (NoSuchElementException, TimeoutException):
                image_url = "N/A"
                print(f"Image URL not found for link: {link}")

            description = link

            # 사이즈 정보 가져오기
            sizes = []
            try:
                ul_element = wait.until(EC.presence_of_element_located((By.XPATH, '//ul[contains(@class, "sc-1sxlp32-1") or contains(@class, "sc-8wsa6t-1 Qtsoe")]')))
                li_elements = ul_element.find_elements(By.TAG_NAME, 'li')
                for li in li_elements:
                    size_text = li.text.strip()
                    if size_text not in ["cm", "MY"]:
                        sizes.append(size_text)
            except (NoSuchElementException, TimeoutException):
                print(f"Size information not found for link: {link}")

            # 리뷰 정보 가져오기
            reviews = extract_reviews(driver, wait)

            product = {
                "product_id": product_id,
                "product_name": product_name,
                "category": category,
                "price": price,
                "image_url": image_url,
                "description": description,
                "size": sizes,
                "reviews": reviews
            }
            products.append(product)
            
            # 데이터 출력
            print(product)

        except Exception as e:
            print(f"Failed to extract data for link: {link} due to {e}")
            continue
    
    return products

def save_to_csv(products, filename="product_test.csv"):
    df = pd.DataFrame(products)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Data saved to {filename}")

def main():
    URL = "https://www.musinsa.com/categories/item/001?device=mw&sortCode=emt_high"
    option = webdriver.ChromeOptions()
    option.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)
    driver.get(URL)
    time.sleep(3)

    driver.find_element(By.XPATH, '//*[@id="root"]/main/div/div[3]/div/button').click()
    time.sleep(1)

    wait = WebDriverWait(driver, 10)
    actions = driver.find_element(By.CSS_SELECTOR, 'body')

    href_links = get_href_links(driver, wait, actions, num_items_to_fetch=100)
    print(f"Number of unique href links: {len(href_links)}")

    products = get_product_info(driver, wait, href_links)
    print("Products extracted:")
    for product in products:
        print(product)

    save_to_csv(products, filename="product_test.csv")

    # driver.quit()

if __name__ == '__main__':
    main()


Current number of unique href links: 45
Current number of unique href links: 90
Current number of unique href links: 135
Number of unique href links: 135
review_id: LV.6 세심한뉴욕셔츠
weight_height_gender: 남성 · 171cm · 74kg
top_size: 커요
brightness_comment: 보통이에요
color_comment: 보통이에요
thickness_comment: 보통이에요
purchased_product_id: https://www.musinsa.com/app/goods/1911516/0
purchased_size: 네이비/L
comment: 색감도 선명하고 질감도 매우 우수한 편으로 만족. 단 냄새가 좀 남
review_id: LV.5 지금뜨는은색포멀
weight_height_gender: 여성 · 165cm · 65kg
top_size: 커요
brightness_comment: 보통이에요
color_comment: 보통이에요
thickness_comment: 보통이에요
purchased_product_id: https://www.musinsa.com/app/goods/1911516/0
purchased_size: 그레이/2XL
comment: 지금 입기 딱 좋은 살짝 두께감 있는 옷
긴팔티 레이어드해서 입어요
review_id: LV.5 박일수
weight_height_gender: 남성 · 177cm · 65kg
top_size: 보통이에요
brightness_comment: 보통이에요
color_comment: 보통이에요
thickness_comment: 보통이에요
purchased_product_id: https://www.musinsa.com/app/goods/1911516/0
purchased_size: 네이비/L
comment: 진짜 너무너무 예쁩니다 후회라곤 1도 없는 제품입니다.

KeyboardInterrupt: 

In [1]:
# 멀티쓰레드 테스트
'''
쓰레드 테스트 결과 90분에서 40분으로 단축
이제 남은 것은 color 정보를 가져오기만 하면됨.

color 정보는 고려해봐야할 듯 함.

'''
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import concurrent.futures

def get_href_links(driver, wait, actions, num_items_to_fetch=100):
    href_links = set()
    while len(href_links) < num_items_to_fetch:
        actions.send_keys(Keys.END)
        time.sleep(2)
        
        for x in range(1, 46):
            try:
                xpath = f'//*[@id="root"]/main/div/section[3]/div[1]/div/div[{x}]/div/div[2]/a[2]'
                element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
                href = element.get_attribute('href')
                href_links.add(href)
            except (NoSuchElementException, TimeoutException):
                continue

        print(f"Current number of unique href links: {len(href_links)}")
        if len(href_links) >= num_items_to_fetch:
            break

    return list(href_links)

def extract_reviews(driver, wait):
    actions = driver.find_element(By.CSS_SELECTOR, 'body')
    time.sleep(1)
    actions.send_keys(Keys.END)
    time.sleep(2)
    actions.send_keys(Keys.END)
    time.sleep(2)
    actions.send_keys(Keys.END)
    time.sleep(1)

    reviews = []
    for n in range(3, 10):
        try:
            review_id = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[1]/div/div[1]/p[1]'))).text
            print(f"review_id: {review_id}")
            
            try:
                weight_height_gender = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[1]/div/div[2]/p[1]'))).text
                print(f"weight_height_gender: {weight_height_gender}")
            except:
                weight_height_gender = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[1]/div/div[2]/p'))).text
                print(f"weight_height_gender (alternative): {weight_height_gender}")
            
            top_size = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[1]/span'))).text
            print(f"top_size: {top_size}")
            
            brightness_comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[2]/span'))).text
            print(f"brightness_comment: {brightness_comment}")
            
            color_comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[3]/span'))).text
            print(f"color_comment: {color_comment}")
            
            thickness_comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[4]/span'))).text
            print(f"thickness_comment: {thickness_comment}")
            
            purchased_product_id = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[2]/div[2]/a'))).get_attribute('href')
            print(f"purchased_product_id: {purchased_product_id}")
            
            purchased_size = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[2]/div[2]/p/span'))).text
            print(f"purchased_size: {purchased_size}")
            
            comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[2]'))).text
            print(f"comment: {comment}")

            review = {
                "weight_height_gender": weight_height_gender,
                "review_id": review_id,
                "top_size": top_size,
                "brightness_comment": brightness_comment,
                "color_comment": color_comment,
                "thickness_comment": thickness_comment,
                "purchased_product_id": purchased_product_id,
                "purchased_size": purchased_size,
                "comment": comment
            }
            reviews.append(review)
        except (NoSuchElementException, TimeoutException):
            print(f"Review information not found for element index: {n}")
            continue
    
    return reviews

def get_product_info(driver, wait, href_links):
    products = []
    for index, link in enumerate(href_links):
        driver.get(link)
        time.sleep(2)  # 페이지 로드 대기

        try:
            product_id = f"top{index + 1}"
            
            try:
                product_name = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[3]/h2'))).text
            except (NoSuchElementException, TimeoutException):
                product_name = "N/A"
                print(f"Product name not found for link: {link}")
                
            try:
                category = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[2]/a[1]'))).text
            except (NoSuchElementException, TimeoutException):
                category = "N/A"
                print(f"Category not found for link: {link}")

            try:
                price = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[5]/div/div/span'))).text.strip()
            except (NoSuchElementException, TimeoutException):
                price = "N/A"
                print(f"Price not found for link: {link}")

            try:
                image_element = wait.until(EC.presence_of_element_located((By.XPATH, '//img[@class="sc-1jl6n79-4 AqRjD"]')))
                image_url = image_element.get_attribute('src')
            except (NoSuchElementException, TimeoutException):
                image_url = "N/A"
                print(f"Image URL not found for link: {link}")

            description = link

            sizes = []
            try:
                ul_element = wait.until(EC.presence_of_element_located((By.XPATH, '//ul[contains(@class, "sc-1sxlp32-1") or contains(@class, "sc-8wsa6t-1 Qtsoe")]')))
                li_elements = ul_element.find_elements(By.TAG_NAME, 'li')
                for li in li_elements:
                    size_text = li.text.strip()
                    if size_text not in ["cm", "MY"]:
                        sizes.append(size_text)
            except (NoSuchElementException, TimeoutException):
                print(f"Size information not found for link: {link}")

            reviews = extract_reviews(driver, wait)

            product = {
                "product_id": product_id,
                "product_name": product_name,
                "category": category,
                "price": price,
                "image_url": image_url,
                "description": description,
                "size": sizes,
                "reviews": reviews
            }
            products.append(product)
            
            print(product)

        except Exception as e:
            print(f"Failed to extract data for link: {link} due to {e}")
            continue
    
    return products

def save_to_csv(products, filename="products.csv"):
    df = pd.DataFrame(products)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Data saved to {filename}")

def split_list(lst, n):
    k, m = divmod(len(lst), n)
    return [lst[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n)]

def process_links(links):
    option = webdriver.ChromeOptions()
    option.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)
    wait = WebDriverWait(driver, 10)

    products = get_product_info(driver, wait, links)
    driver.quit()
    return products

def main():
    URL = "https://www.musinsa.com/categories/item/001?device=mw&sortCode=emt_high"
    option = webdriver.ChromeOptions()
    option.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)
    driver.get(URL)
    time.sleep(3)

    driver.find_element(By.XPATH, '//*[@id="root"]/main/div/div[3]/div/button').click()
    time.sleep(1)

    wait = WebDriverWait(driver, 10)
    actions = driver.find_element(By.CSS_SELECTOR, 'body')

    href_links = get_href_links(driver, wait, actions, num_items_to_fetch=100)
    driver.quit()

    sub_lists = split_list(href_links, 4)
    # Thread info
    # https://velog.io/@cha-suyeon/%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%97%90%EC%84%9C-%EC%8A%A4%EB%A0%88%EB%93%9C%ED%94%84%EB%A1%9C%EC%84%B8%EC%8A%A4-%ED%92%80-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        future_to_links = {executor.submit(process_links, sub_list): sub_list for sub_list in sub_lists}
        all_products = []
        for future in concurrent.futures.as_completed(future_to_links):
            products = future.result()
            all_products.extend(products)

    save_to_csv(all_products)

if __name__ == '__main__':
    main()


Current number of unique href links: 45
Current number of unique href links: 90
Current number of unique href links: 135
review_id: LV.4 18888
weight_height_gender: 여성 · 165cm · 47kg
top_size: 보통이에요
review_id: LV.7 송장개구리
brightness_comment: 보통이에요
weight_height_gender: 남성 · 172cm · 73kg
color_comment: 보통이에요
top_size: 보통이에요
thickness_comment: 얇아요
brightness_comment: 보통이에요
purchased_product_id: https://www.musinsa.com/app/goods/1417716/0
color_comment: 선명해요
purchased_size: S
thickness_comment: 얇아요
comment: 딱 제가 원했던 기본 크롭티에요. 길이도 적당합니다
purchased_product_id: https://www.musinsa.com/app/goods/1388775/0
review_id: LV.4 능엇믹
purchased_size: L
weight_height_gender: 여성 · 160cm · 41kg
comment: 일단 두께감이 얇아서 착용감도 편하고 시원합니다.
가성비가 정말 좋네요
밝은 컬러는 잘 모르겠지만
어두운 컬러라서 그런지 비침은 없네요!
대만족입니다
top_size: 보통이에요
review_id: LV.6 cksean
brightness_comment: 보통이에요
weight_height_gender: 남성 · 182cm · 80kg
color_comment: 선명해요
top_size: 보통이에요
thickness_comment: 보통이에요
brightness_comment: 어두워요
purchased_product_id: https://www.

In [ ]:
# Thread count share update
'''
쓰레드 테스트 결과 90분에서 40분으로 단축
이제 남은 것은 color 정보를 가져오기만 하면됨.

color 정보는 고려해봐야할 듯 함.

+

csv파일 조회 결과 쓰레드 간 카운트를 공유하지 않는 문제 발생.
이를 해결하기 위해 수정.


'''
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import concurrent.futures
from multiprocessing import Manager

def get_href_links(driver, wait, actions, num_items_to_fetch=100):
    href_links = set()
    while len(href_links) < num_items_to_fetch:
        actions.send_keys(Keys.END)
        time.sleep(2)
        
        for x in range(1, 46):
            try:
                xpath = f'//*[@id="root"]/main/div/section[3]/div[1]/div/div[{x}]/div/div[2]/a[2]'
                element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
                href = element.get_attribute('href')
                href_links.add(href)
            except (NoSuchElementException, TimeoutException):
                continue

        print(f"Current number of unique href links: {len(href_links)}")
        if len(href_links) >= num_items_to_fetch:
            break

    return list(href_links)

def extract_reviews(driver, wait):
    actions = driver.find_element(By.CSS_SELECTOR, 'body')
    time.sleep(1)
    actions.send_keys(Keys.END)
    time.sleep(2)
    actions.send_keys(Keys.END)
    time.sleep(2)
    actions.send_keys(Keys.END)
    time.sleep(1)

    reviews = []
    for n in range(3, 10):
        try:
            review_id = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[1]/div/div[1]/p[1]'))).text
            print(f"review_id: {review_id}")
            
            try:
                weight_height_gender = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[1]/div/div[2]/p[1]'))).text
                print(f"weight_height_gender: {weight_height_gender}")
            except:
                weight_height_gender = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[1]/div/div[2]/p'))).text
                print(f"weight_height_gender (alternative): {weight_height_gender}")
            
            top_size = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[1]/span'))).text
            print(f"top_size: {top_size}")
            
            brightness_comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[2]/span'))).text
            print(f"brightness_comment: {brightness_comment}")
            
            color_comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[3]/span'))).text
            print(f"color_comment: {color_comment}")
            
            thickness_comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[1]/ul/li[4]/span'))).text
            print(f"thickness_comment: {thickness_comment}")
            
            purchased_product_id = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[2]/div[2]/a'))).get_attribute('href')
            print(f"purchased_product_id: {purchased_product_id}")
            
            purchased_size = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[2]/div[2]/p/span'))).text
            print(f"purchased_size: {purchased_size}")
            
            comment = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="style_estimate_list"]/div/div[{n}]/div[4]/div[2]'))).text
            print(f"comment: {comment}")

            review = {
                "weight_height_gender": weight_height_gender,
                "review_id": review_id,
                "top_size": top_size,
                "brightness_comment": brightness_comment,
                "color_comment": color_comment,
                "thickness_comment": thickness_comment,
                "purchased_product_id": purchased_product_id,
                "purchased_size": purchased_size,
                "comment": comment
            }
            reviews.append(review)
        except (NoSuchElementException, TimeoutException):
            print(f"Review information not found for element index: {n}")
            continue
    
    return reviews

def get_product_info(driver, wait, href_links, count):
    products = []
    for index, link in enumerate(href_links):
        driver.get(link)
        time.sleep(2)

        try:
            product_id = f"top{index + 1}"
            
            try:
                product_name = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[3]/h2'))).text
            except (NoSuchElementException, TimeoutException):
                product_name = "N/A"
                print(f"Product name not found for link: {link}")
                
            try:
                category = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[2]/a[1]'))).text
            except (NoSuchElementException, TimeoutException):
                category = "N/A"
                print(f"Category not found for link: {link}")

            try:
                price = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[5]/div/div/span'))).text.strip()
            except (NoSuchElementException, TimeoutException):
                price = "N/A"
                print(f"Price not found for link: {link}")

            try:
                image_element = wait.until(EC.presence_of_element_located((By.XPATH, '//img[@class="sc-1jl6n79-4 AqRjD"]')))
                image_url = image_element.get_attribute('src')
            except (NoSuchElementException, TimeoutException):
                image_url = "N/A"
                print(f"Image URL not found for link: {link}")

            description = link

            sizes = []
            try:
                ul_element = wait.until(EC.presence_of_element_located((By.XPATH, '//ul[contains(@class, "sc-1sxlp32-1") or contains(@class, "sc-8wsa6t-1 Qtsoe")]')))
                li_elements = ul_element.find_elements(By.TAG_NAME, 'li')
                for li in li_elements:
                    size_text = li.text.strip()
                    if size_text not in ["cm", "MY"]:
                        sizes.append(size_text)
            except (NoSuchElementException, TimeoutException):
                print(f"Size information not found for link: {link}")

            reviews = extract_reviews(driver, wait)

            product = {
                "product_id": product_id,
                "product_name": product_name,
                "category": category,
                "price": price,
                "image_url": image_url,
                "description": description,
                "size": sizes,
                "reviews": reviews
            }
            products.append(product)
            
            print(product)

        except Exception as e:
            print(f"Failed to extract data for link: {link} due to {e}")
            continue

        with count.get_lock():
            count.value += 1
            print(f"Processed {count.value} products")

    return products

def save_to_csv(products, filename="products.csv"):
    df = pd.DataFrame(products)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Data saved to {filename}")

def split_list(lst, n):
    k, m = divmod(len(lst), n)
    return [lst[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n)]

def process_links(links, count):
    option = webdriver.ChromeOptions()
    option.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)
    wait = WebDriverWait(driver, 10)

    products = get_product_info(driver, wait, links, count)
    driver.quit()
    return products

def main():
    URL = "https://www.musinsa.com/categories/item/001?device=mw&sortCode=emt_high"
    option = webdriver.ChromeOptions()
    option.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)
    driver.get(URL)
    time.sleep(3)

    driver.find_element(By.XPATH, '//*[@id="root"]/main/div/div[3]/div/button').click()
    time.sleep(1)

    wait = WebDriverWait(driver, 10)
    actions = driver.find_element(By.CSS_SELECTOR, 'body')

    href_links = get_href_links(driver, wait, actions, num_items_to_fetch=100)
    driver.quit()

    sub_lists = split_list(href_links, 4)

    manager = Manager()
    count = manager.Value('i', 0)

    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(process_links, sub_list, count) for sub_list in sub_lists]
        all_products = []
        for future in concurrent.futures.as_completed(futures):
            products = future.result()
            all_products.extend(products)

    save_to_csv(all_products)

if __name__ == '__main__':
    main()


현재까지 color 정보를 제외한 모든 정보를 가져올 수 있음

그러면 남은 과제는 무엇일까?

color 정보를 가져와야 함.

기존에 csv에서 description에서 이미 존재한 상품 링크이면 조회하지 않고 새로운 것만 "추가"하는 방식으로 가야함. (업데이트 시 시간 단축)

나중에 s3연동이 되면 s3로 업로드를 하는 방식으로 가야함 (링크 조회는 아마 데이터 웨어하우스에서하거나 s3를 쿼리처럼 하는 방식처럼 이용)

주기적으로 실행하기 위해 airflow dag code로 transform해야함.

[데이터 전처리 파트]
product_id,product_name,category,price,image_url,description,size,reviews 

현재 코드인데 reviews는 또 안에 array 형태로 있고 해서 분리작업이 필요하긴하다.
